In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import joblib

# check xgboost version
from xgboost import XGBRegressor

#torch stuff
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#from torch_geometric.utils import dense_to_sparse


import lightgbm as lgb
from lightgbm import LGBMRegressor

import holidays
us_holidays = holidays.country_holidays('US')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [5]:
df = pd.read_csv(f'{data_dir}/final_model_input_partial_scale_3.csv')

df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])
df = df.sort_values(by=['start_station_cluster', 'started_at_hourly'])

df_test = df[pd.to_datetime(
    df['started_at_hourly']) >= pd.to_datetime("2024-01-01 00:00:00")]
df_train = df[pd.to_datetime(
    df['started_at_hourly']) < pd.to_datetime("2024-01-01 00:00:00")]

del df

In [6]:
target_scaler = joblib.load(f'{models_dir}/target_scaler.sav')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
execfile('/content/drive/MyDrive/Thesis/models_training/model_training.py')

Using device: cpu


# GNN Mean-Variance Embedding

In [8]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            'is_holiday',
            #'flag_added',
            'temp',
            'dwpt',
            'rhum',
            'prcp',
            'wdir',
            'wspd',
            'pres',
            'coco',
            "demand_lag_1_h",
            "demand_lag_2_h",
            "demand_lag_24_h",
            #"temp_lag_1_h",
            #"temp_lag_2_h",
            #"temp_lag_24_h",
            #"prcp_lag_1_h",
            #"prcp_lag_2_h",
            #"prcp_lag_24_h",
            #"rhum_lag_1_h",
            #"rhum_lag_2_h",
            #"rhum_lag_24_h",
            #"wspd_lag_1_h",
            #"wspd_lag_2_h",
            #"wspd_lag_24_h",
            "total_demand_1h",
            "demand_degrees_1h",
]
FEATURES.extend([f"dim_var_{i}" for i in range(50)])
FEATURES.extend([f"dim_mean_{i}" for i in range(50)])

## XGBoost

In [8]:
model, model_preds  = train_model(df_train, df_test, FEATURES, model='xgb')

overall
MSE: 3.660790538947053
RMSE: 1.913319246479022
MAE: 1.0307203251880628
MAPE: 0.4214031077956738

Non-zero
MSE: 7.3958121340257295
RMSE: 2.7195242477363077
MAE: 1.7147152117829707
MAPE: 0.36017576726440015

Zeros
MSE: 0.6069189790633743
RMSE: 0.7790500491389333
MAE: 0.47146451418275814
MAPE: 0.47146451418275814


In [ ]:
#11_01
3.1979816352695654
1.788290142921323
0.3706682703327132

#11-09 scaled
3.649102210215345
1.910262340678721
0.4127466890916304

#11-09 unscaled
3.654871476133676
1.9117718159167627
0.4152402407914972

#11-28
3.660790538947053
1.913319246479022
0.4214031077956738

0.4214031077956738

In [ ]:
suf = "_g_xgb_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## LightGBM

In [9]:
model, model_preds  = train_model(df_train, df_test, FEATURES, model='lgbm')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.069981 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 27056
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 122
[LightGBM] [Info] Start training from score 1.005143
overall
MSE: 3.815821115916084
RMSE: 1.9534126844873523
MAE: 1.0555935431736703
MAPE: 0.4294809796900471

Non-zero
MSE: 7.721836279234099
RMSE: 2.778819223921214
MAE: 1.7579401067117186
MAPE: 0.3660638561493653

Zeros
MSE: 0.6221398168712727
RMSE: 0.7887584020923472
MAE: 0.481332821790692
MAPE: 0.481332821790692


In [ ]:
suf = "_lgbm_rf_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## Random Forest

In [ ]:
model, model_preds  = train_model(df_train, df_test, FEATURES, model='rf')

In [ ]:
#11_01
3.601167851840035
1.897674327127823
0.40947944791286134

#11-28
4.125786924330509
2.0312033192988115
0.4574572447524205

0.40947944791286134

In [ ]:
suf = "_g_rf_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model